In [1]:
import pandas as pd
import numpy as np

In [2]:
import re
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
import unicodedata
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

# Using `https://allafrica.com/`

# TOPIC 1 : Vaccination

In [4]:
data = pd.read_csv(r'C:\Users\ilboudod\CovidPerception\Files\data_vac.csv', sep='\t', encoding='iso-8859-1')

In [5]:
#data.head()

In [6]:
Countries=['ALGERIA', 'ANGOLA', 'BENIN', 'BOTSWANA',
       'BURKINA FASO', 'BURUNDI', 'CAMEROON', 'CABO VERDE',
       'CENTRAL AFRICAN REPUBLIC', 'CHAD', 'COMOROS', 'REPUBLIC OF CONGO',
       "CÔTE D'IVOIRE", 'DEMOCRATIC REPUBLIC OF CONGO',
       'EQUATORIAL GUINEA', 'ERITREA', 'ETHIOPIA', 'GABON', 'GAMBIA',
       'GHANA', 'GUINEA', 'GUINEA-BISSAU', 'KENYA', 'LESOTHO', 'LIBERIA',
       'MADAGASCAR', 'MALAWI', 'MALI', 'MAURITANIA', 'MAURITIUS',
       'MOZAMBIQUE', 'NAMIBIA', 'NIGER', 'NIGERIA', 'RWANDA',
       'SÃO TOMÉ AND PRÍNCIPE', 'SENEGAL', 'SEYCHELLES', 'SIERRA LEONE',
       'SOUTH AFRICA', 'SOUTH SUDAN', 'ESWATINI', 'TOGO', 'UGANDA',
       'UNITED REPUBLIC OF TANZANIA', 'ZAMBIA', 'ZIMBABWE']
CountryCode=['ALG', 'AGO', 'BEN', 'BWA', 'BFA', 'BDI', 'CMR', 'CPV',
            'CAF', 'TCD', 'COM', 'COG', 'CIV', 'DRC', 'GNQ', 'ERI',
            'ETH', 'GAB', 'GMB', 'GHA', 'GIN', 'GNB', 'KEN', 'LSO', 
            'LBR', 'MDG', 'MWI', 'MLI', 'MRT', 'MUS', 'MOZ', 'NAM',
            'NER', 'NGA', 'RWA', 'STP', 'SEN', 'SYC', 'SLE', 'ZAF',
            'SSD', 'SZ', 'TGO', 'UGA', 'TZA', 'ZMB', 'ZW']

In [7]:
data['TitleL'] = data['Title'].str.lower()
data['StorieL'] = data['Storie'].str.lower()
CountryWithData = []
CountryWithDataIndex = []
CountryWithDatalen = []
for country, code, i in zip(Countries, CountryCode, range(len(Countries))):
    country = country.lower()
    dtCountry = data[(data['StorieL'].str.contains(country)) | (data['TitleL'].str.contains(country))]
    if (len(dtCountry)>0):
        CountryWithData.append(Countries[i])
        CountryWithDataIndex.append(i)
        CountryWithDatalen.append(len(dtCountry))
        globals()["data_vac_" + code] = dtCountry

#### Cleaning Content

In [8]:
#Remove Punctuation
def remove_punct(txt):
    txt  = "".join([char for char in txt if char not in string.punctuation])    
    txt1 = re.sub('[0-9]+', '', txt)
    #remove mentions@
    txt1 = re.sub(r'@[A-Za-z0-9_]+', '', txt1)
    # Remove hashtags
    txt1 = re.sub(r'#', '', txt1)
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    no_url = url_pattern.sub(r'', txt1)
    text = (unicodedata.normalize('NFKD', no_url).encode('ascii', 'ignore').decode('utf-8', 'ignore'))
    return text

In [9]:
def SentimentAnalysisCountry(data, var, threshold):
    
    sid = SIA()
    data['sentiments'] = data[var].apply(lambda x: sid.polarity_scores(' '.join(re.findall(r'\w+',x))))

    # Obtaining NLTK compound score
    data['nltk_cmp_score'] = data['sentiments'].apply(lambda score_dict: score_dict['compound'])


    data['Positive Sentiment'] = data['sentiments'].apply(lambda x: x['pos']+1*(10**-6)) 
    data['Neutral Sentiment'] = data['sentiments'].apply(lambda x: x['neu']+1*(10**-6))
    data['Negative Sentiment'] = data['sentiments'].apply(lambda x: x['neg']+1*(10**-6))
    
    neutral_thresh = threshold
    
    # Categorize scores into the sentiments of positive, neutral or negative using compund
    data['nltk_sentiment'] = data['nltk_cmp_score'].apply(lambda c: 'Positive' if c >= neutral_thresh else ('Negative' if c <= -(neutral_thresh) else 'Neutral'))
    return data

In [10]:
# Define function to get value counts
def get_value_counts(data, col_name):
    count = pd.DataFrame(data[col_name].value_counts())
    percentage = pd.DataFrame(data[col_name].value_counts(normalize=True).mul(100))
    value_counts_df = pd.concat([count, percentage], axis = 1)
    value_counts_df = value_counts_df.reset_index()
    value_counts_df.columns = ['sentiment', 'counts', 'percentage']
    value_counts_df.sort_values('sentiment', inplace = True)
    value_counts_df['percentage'] = value_counts_df['percentage'].apply(lambda x: round(x,2))
    value_counts_df = value_counts_df.reset_index(drop = True)
    return value_counts_df

In [12]:
import nltk

In [13]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\ilboudod\AppData\Roaming\nltk_data...


True

In [14]:
#Algerie
data = data_vac_ALG.copy()
data["Content"] = data[['Title', 'Storie']].apply(lambda x: ' '.join(x), axis=1)
data = data[data.Storie.notna()]
data = data[data.Content.notna()]
data['Content_punct'] = data['Content'].apply(lambda x:remove_punct(x.lower()))
data_final = SentimentAnalysisCountry(data, 'Content_punct', 0.05)
nltk_sentiment_df = get_value_counts(data_final, 'nltk_sentiment')
nltk_sentiment_df

,sentiment,counts,percentage
0,Negative,58,16.57
1,Neutral,1,0.29
2,Positive,291,83.14


In [11]:
#ANGOLA
data = data_vac_AGO.copy()
data["Content"] = data[['Title', 'Storie']].apply(lambda x: ' '.join(x), axis=1)
data = data[data.Storie.notna()]
data = data[data.Content.notna()]
data['Content_punct'] = data['Content'].apply(lambda x:remove_punct(x.lower()))
data_final = SentimentAnalysisCountry(data, 'Content_punct', 0.05)
nltk_sentiment_df = get_value_counts(data_final, 'nltk_sentiment')
nltk_sentiment_df

,sentiment,counts,percentage
0,Negative,181,20.11
1,Neutral,12,1.33
2,Positive,707,78.56


In [12]:
#BENIN
data = data_vac_BEN.copy()
data["Content"] = data[['Title', 'Storie']].apply(lambda x: ' '.join(x), axis=1)
data = data[data.Storie.notna()]
data = data[data.Content.notna()]
data['Content_punct'] = data['Content'].apply(lambda x:remove_punct(x.lower()))
data_final = SentimentAnalysisCountry(data, 'Content_punct', 0.05)
nltk_sentiment_df = get_value_counts(data_final, 'nltk_sentiment')
nltk_sentiment_df

,sentiment,counts,percentage
0,Negative,87,25.07
1,Positive,260,74.93


In [13]:
#BOTSWANA
data = data_vac_BWA.copy()
data["Content"] = data[['Title', 'Storie']].apply(lambda x: ' '.join(x), axis=1)
data = data[data.Storie.notna()]
data = data[data.Content.notna()]
data['Content_punct'] = data['Content'].apply(lambda x:remove_punct(x.lower()))
data_final = SentimentAnalysisCountry(data, 'Content_punct', 0.05)
nltk_sentiment_df = get_value_counts(data_final, 'nltk_sentiment')
nltk_sentiment_df

,sentiment,counts,percentage
0,Negative,179,17.50
1,Neutral,2,0.20
2,Positive,842,82.31


In [14]:
#BURKINA FASO
data = data_vac_BFA.copy()
data["Content"] = data[['Title', 'Storie']].apply(lambda x: ' '.join(x), axis=1)
data = data[data.Storie.notna()]
data = data[data.Content.notna()]
data['Content_punct'] = data['Content'].apply(lambda x:remove_punct(x.lower()))
data_final = SentimentAnalysisCountry(data, 'Content_punct', 0.05)
nltk_sentiment_df = get_value_counts(data_final, 'nltk_sentiment')
nltk_sentiment_df

,sentiment,counts,percentage
0,Negative,98,30.63
1,Positive,222,69.38


In [15]:
#BURUNDI
data = data_vac_BDI.copy()
data["Content"] = data[['Title', 'Storie']].apply(lambda x: ' '.join(x), axis=1)
data = data[data.Storie.notna()]
data = data[data.Content.notna()]
data['Content_punct'] = data['Content'].apply(lambda x:remove_punct(x.lower()))
data_final = SentimentAnalysisCountry(data, 'Content_punct', 0.05)
nltk_sentiment_df = get_value_counts(data_final, 'nltk_sentiment')
nltk_sentiment_df

,sentiment,counts,percentage
0,Negative,86,24.64
1,Positive,263,75.36


In [16]:
#CAMEROON
data = data_vac_CMR.copy()
data["Content"] = data[['Title', 'Storie']].apply(lambda x: ' '.join(x), axis=1)
data = data[data.Storie.notna()]
data = data[data.Content.notna()]
data['Content_punct'] = data['Content'].apply(lambda x:remove_punct(x.lower()))
data_final = SentimentAnalysisCountry(data, 'Content_punct', 0.05)
nltk_sentiment_df = get_value_counts(data_final, 'nltk_sentiment')
nltk_sentiment_df

,sentiment,counts,percentage
0,Negative,153,20.73
1,Neutral,1,0.14
2,Positive,584,79.13


In [17]:
#CABO VERDE
data = data_vac_CPV.copy()
data["Content"] = data[['Title', 'Storie']].apply(lambda x: ' '.join(x), axis=1)
data = data[data.Storie.notna()]
data = data[data.Content.notna()]
data['Content_punct'] = data['Content'].apply(lambda x:remove_punct(x.lower()))
data_final = SentimentAnalysisCountry(data, 'Content_punct', 0.05)
nltk_sentiment_df = get_value_counts(data_final, 'nltk_sentiment')
nltk_sentiment_df

,sentiment,counts,percentage
0,Negative,13,18.06
1,Positive,59,81.94


In [18]:
#CENTRAL AFRICAN REPUBLIC
data = data_vac_CAF.copy()
data["Content"] = data[['Title', 'Storie']].apply(lambda x: ' '.join(x), axis=1)
data = data[data.Storie.notna()]
data = data[data.Content.notna()]
data['Content_punct'] = data['Content'].apply(lambda x:remove_punct(x.lower()))
data_final = SentimentAnalysisCountry(data, 'Content_punct', 0.05)
nltk_sentiment_df = get_value_counts(data_final, 'nltk_sentiment')
nltk_sentiment_df

,sentiment,counts,percentage
0,Negative,67,41.36
1,Positive,95,58.64


In [19]:
#CHAD
data = data_vac_TCD.copy()
data["Content"] = data[['Title', 'Storie']].apply(lambda x: ' '.join(x), axis=1)
data = data[data.Storie.notna()]
data = data[data.Content.notna()]
data['Content_punct'] = data['Content'].apply(lambda x:remove_punct(x.lower()))
data_final = SentimentAnalysisCountry(data, 'Content_punct', 0.05)
nltk_sentiment_df = get_value_counts(data_final, 'nltk_sentiment')
nltk_sentiment_df

,sentiment,counts,percentage
0,Negative,157,36.85
1,Neutral,2,0.47
2,Positive,267,62.68


In [20]:
#COMOROS
data = data_vac_COM.copy()
data["Content"] = data[['Title', 'Storie']].apply(lambda x: ' '.join(x), axis=1)
data = data[data.Storie.notna()]
data = data[data.Content.notna()]
data['Content_punct'] = data['Content'].apply(lambda x:remove_punct(x.lower()))
data_final = SentimentAnalysisCountry(data, 'Content_punct', 0.05)
nltk_sentiment_df = get_value_counts(data_final, 'nltk_sentiment')
nltk_sentiment_df

,sentiment,counts,percentage
0,Negative,27,18.75
1,Positive,117,81.25


In [21]:
#REPUBLIC OF CONGO
data = data_vac_COG.copy()
data["Content"] = data[['Title', 'Storie']].apply(lambda x: ' '.join(x), axis=1)
data = data[data.Storie.notna()]
data = data[data.Content.notna()]
data['Content_punct'] = data['Content'].apply(lambda x:remove_punct(x.lower()))
data_final = SentimentAnalysisCountry(data, 'Content_punct', 0.05)
nltk_sentiment_df = get_value_counts(data_final, 'nltk_sentiment')
nltk_sentiment_df

,sentiment,counts,percentage
0,Negative,125,28.03
1,Neutral,1,0.22
2,Positive,320,71.75


In [22]:
#CÔTE D'IVOIRE
data = data_vac_CIV.copy()
data["Content"] = data[['Title', 'Storie']].apply(lambda x: ' '.join(x), axis=1)
data = data[data.Storie.notna()]
data = data[data.Content.notna()]
data['Content_punct'] = data['Content'].apply(lambda x:remove_punct(x.lower()))
data_final = SentimentAnalysisCountry(data, 'Content_punct', 0.05)
nltk_sentiment_df = get_value_counts(data_final, 'nltk_sentiment')
nltk_sentiment_df

NameError: name 'data_vac_CIV' is not defined

In [23]:
#DEMOCRATIC REPUBLIC OF CONGO
data = data_vac_DRC.copy()
data["Content"] = data[['Title', 'Storie']].apply(lambda x: ' '.join(x), axis=1)
data = data[data.Storie.notna()]
data = data[data.Content.notna()]
data['Content_punct'] = data['Content'].apply(lambda x:remove_punct(x.lower()))
data_final = SentimentAnalysisCountry(data, 'Content_punct', 0.05)
nltk_sentiment_df = get_value_counts(data_final, 'nltk_sentiment')
nltk_sentiment_df

,sentiment,counts,percentage
0,Negative,114,27.94
1,Neutral,1,0.25
2,Positive,293,71.81


In [24]:
#EQUATORIAL GUINEA
data = data_vac_GNQ.copy()
data["Content"] = data[['Title', 'Storie']].apply(lambda x: ' '.join(x), axis=1)
data = data[data.Storie.notna()]
data = data[data.Content.notna()]
data['Content_punct'] = data['Content'].apply(lambda x:remove_punct(x.lower()))
data_final = SentimentAnalysisCountry(data, 'Content_punct', 0.05)
nltk_sentiment_df = get_value_counts(data_final, 'nltk_sentiment')
nltk_sentiment_df

,sentiment,counts,percentage
0,Negative,20,21.51
1,Positive,73,78.49


In [25]:
#ERITREA
data = data_vac_ERI.copy()
data["Content"] = data[['Title', 'Storie']].apply(lambda x: ' '.join(x), axis=1)
data = data[data.Storie.notna()]
data = data[data.Content.notna()]
data['Content_punct'] = data['Content'].apply(lambda x:remove_punct(x.lower()))
data_final = SentimentAnalysisCountry(data, 'Content_punct', 0.05)
nltk_sentiment_df = get_value_counts(data_final, 'nltk_sentiment')
nltk_sentiment_df

,sentiment,counts,percentage
0,Negative,71,16.51
1,Neutral,1,0.23
2,Positive,358,83.26


In [26]:
#ETHIOPIA
data = data_vac_ETH.copy()
data["Content"] = data[['Title', 'Storie']].apply(lambda x: ' '.join(x), axis=1)
data = data[data.Storie.notna()]
data = data[data.Content.notna()]
data['Content_punct'] = data['Content'].apply(lambda x:remove_punct(x.lower()))
data_final = SentimentAnalysisCountry(data, 'Content_punct', 0.05)
nltk_sentiment_df = get_value_counts(data_final, 'nltk_sentiment')
nltk_sentiment_df

,sentiment,counts,percentage
0,Negative,584,37.70
1,Neutral,6,0.39
2,Positive,959,61.91


In [27]:
#GABON
data = data_vac_GAB.copy()
data["Content"] = data[['Title', 'Storie']].apply(lambda x: ' '.join(x), axis=1)
data = data[data.Storie.notna()]
data = data[data.Content.notna()]
data['Content_punct'] = data['Content'].apply(lambda x:remove_punct(x.lower()))
data_final = SentimentAnalysisCountry(data, 'Content_punct', 0.05)
nltk_sentiment_df = get_value_counts(data_final, 'nltk_sentiment')
nltk_sentiment_df

,sentiment,counts,percentage
0,Negative,22,17.05
1,Neutral,1,0.78
2,Positive,106,82.17


In [28]:
#GAMBIA
data = data_vac_GMB.copy()
data["Content"] = data[['Title', 'Storie']].apply(lambda x: ' '.join(x), axis=1)
data = data[data.Storie.notna()]
data = data[data.Content.notna()]
data['Content_punct'] = data['Content'].apply(lambda x:remove_punct(x.lower()))
data_final = SentimentAnalysisCountry(data, 'Content_punct', 0.05)
nltk_sentiment_df = get_value_counts(data_final, 'nltk_sentiment')
nltk_sentiment_df

,sentiment,counts,percentage
0,Negative,144,21.69
1,Neutral,4,0.60
2,Positive,516,77.71


In [29]:
#GHANA
data = data_vac_GHA.copy()
data["Content"] = data[['Title', 'Storie']].apply(lambda x: ' '.join(x), axis=1)
data = data[data.Storie.notna()]
data = data[data.Content.notna()]
data['Content_punct'] = data['Content'].apply(lambda x:remove_punct(x.lower()))
data_final = SentimentAnalysisCountry(data, 'Content_punct', 0.05)
nltk_sentiment_df = get_value_counts(data_final, 'nltk_sentiment')
nltk_sentiment_df

,sentiment,counts,percentage
0,Negative,245,15.78
1,Neutral,8,0.52
2,Positive,1300,83.71


In [30]:
#GUINEA
data = data_vac_GIN.copy()
data["Content"] = data[['Title', 'Storie']].apply(lambda x: ' '.join(x), axis=1)
data = data[data.Storie.notna()]
data = data[data.Content.notna()]
data['Content_punct'] = data['Content'].apply(lambda x:remove_punct(x.lower()))
data_final = SentimentAnalysisCountry(data, 'Content_punct', 0.05)
nltk_sentiment_df = get_value_counts(data_final, 'nltk_sentiment')
nltk_sentiment_df

,sentiment,counts,percentage
0,Negative,133,23.21
1,Neutral,5,0.87
2,Positive,435,75.92


In [31]:
#GUINEA-BISSAU
data = data_vac_GNB.copy()
data["Content"] = data[['Title', 'Storie']].apply(lambda x: ' '.join(x), axis=1)
data = data[data.Storie.notna()]
data = data[data.Content.notna()]
data['Content_punct'] = data['Content'].apply(lambda x:remove_punct(x.lower()))
data_final = SentimentAnalysisCountry(data, 'Content_punct', 0.05)
nltk_sentiment_df = get_value_counts(data_final, 'nltk_sentiment')
nltk_sentiment_df

,sentiment,counts,percentage
0,Negative,6,13.64
1,Positive,38,86.36


In [32]:
#KENYA
data = data_vac_KEN.copy()
data["Content"] = data[['Title', 'Storie']].apply(lambda x: ' '.join(x), axis=1)
data = data[data.Storie.notna()]
data = data[data.Content.notna()]
data['Content_punct'] = data['Content'].apply(lambda x:remove_punct(x.lower()))
data_final = SentimentAnalysisCountry(data, 'Content_punct', 0.05)
nltk_sentiment_df = get_value_counts(data_final, 'nltk_sentiment')
nltk_sentiment_df

,sentiment,counts,percentage
0,Negative,1618,30.33
1,Neutral,33,0.62
2,Positive,3683,69.05


In [33]:
#LESOTHO
data = data_vac_LSO.copy()
data["Content"] = data[['Title', 'Storie']].apply(lambda x: ' '.join(x), axis=1)
data = data[data.Storie.notna()]
data = data[data.Content.notna()]
data['Content_punct'] = data['Content'].apply(lambda x:remove_punct(x.lower()))
data_final = SentimentAnalysisCountry(data, 'Content_punct', 0.05)
nltk_sentiment_df = get_value_counts(data_final, 'nltk_sentiment')
nltk_sentiment_df

,sentiment,counts,percentage
0,Negative,104,27.08
1,Neutral,3,0.78
2,Positive,277,72.14


In [34]:
#LIBERIA
data = data_vac_LBR.copy()
data["Content"] = data[['Title', 'Storie']].apply(lambda x: ' '.join(x), axis=1)
data = data[data.Storie.notna()]
data = data[data.Content.notna()]
data['Content_punct'] = data['Content'].apply(lambda x:remove_punct(x.lower()))
data_final = SentimentAnalysisCountry(data, 'Content_punct', 0.05)
nltk_sentiment_df = get_value_counts(data_final, 'nltk_sentiment')
nltk_sentiment_df

,sentiment,counts,percentage
0,Negative,260,22.53
1,Neutral,1,0.09
2,Positive,893,77.38


In [35]:
#MADAGASCAR
data = data_vac_MDG.copy()
data["Content"] = data[['Title', 'Storie']].apply(lambda x: ' '.join(x), axis=1)
data = data[data.Storie.notna()]
data = data[data.Content.notna()]
data['Content_punct'] = data['Content'].apply(lambda x:remove_punct(x.lower()))
data_final = SentimentAnalysisCountry(data, 'Content_punct', 0.05)
nltk_sentiment_df = get_value_counts(data_final, 'nltk_sentiment')
nltk_sentiment_df

,sentiment,counts,percentage
0,Negative,61,27.6
1,Positive,160,72.4


In [36]:
#MALAWI
data = data_vac_MWI.copy()
data["Content"] = data[['Title', 'Storie']].apply(lambda x: ' '.join(x), axis=1)
data = data[data.Storie.notna()]
data = data[data.Content.notna()]
data['Content_punct'] = data['Content'].apply(lambda x:remove_punct(x.lower()))
data_final = SentimentAnalysisCountry(data, 'Content_punct', 0.05)
nltk_sentiment_df = get_value_counts(data_final, 'nltk_sentiment')
nltk_sentiment_df

,sentiment,counts,percentage
0,Negative,274,24.86
1,Neutral,6,0.54
2,Positive,822,74.59


In [37]:
#MALI
data = data_vac_MLI.copy()
data["Content"] = data[['Title', 'Storie']].apply(lambda x: ' '.join(x), axis=1)
data = data[data.Storie.notna()]
data = data[data.Content.notna()]
data['Content_punct'] = data['Content'].apply(lambda x:remove_punct(x.lower()))
data_final = SentimentAnalysisCountry(data, 'Content_punct', 0.05)
nltk_sentiment_df = get_value_counts(data_final, 'nltk_sentiment')
nltk_sentiment_df

,sentiment,counts,percentage
0,Negative,543,32.26
1,Neutral,3,0.18
2,Positive,1137,67.56


In [38]:
#MAURITANIA
data = data_vac_MRT.copy()
data["Content"] = data[['Title', 'Storie']].apply(lambda x: ' '.join(x), axis=1)
data = data[data.Storie.notna()]
data = data[data.Content.notna()]
data['Content_punct'] = data['Content'].apply(lambda x:remove_punct(x.lower()))
data_final = SentimentAnalysisCountry(data, 'Content_punct', 0.05)
nltk_sentiment_df = get_value_counts(data_final, 'nltk_sentiment')
nltk_sentiment_df

,sentiment,counts,percentage
0,Negative,31,22.14
1,Positive,109,77.86


In [39]:
#MAURITIUS
data = data_vac_MUS.copy()
data["Content"] = data[['Title', 'Storie']].apply(lambda x: ' '.join(x), axis=1)
data = data[data.Storie.notna()]
data = data[data.Content.notna()]
data['Content_punct'] = data['Content'].apply(lambda x:remove_punct(x.lower()))
data_final = SentimentAnalysisCountry(data, 'Content_punct', 0.05)
nltk_sentiment_df = get_value_counts(data_final, 'nltk_sentiment')
nltk_sentiment_df

,sentiment,counts,percentage
0,Negative,67,11.30
1,Neutral,9,1.52
2,Positive,517,87.18


In [40]:
#MOZAMBIQUE
data = data_vac_MOZ.copy()
data["Content"] = data[['Title', 'Storie']].apply(lambda x: ' '.join(x), axis=1)
data = data[data.Storie.notna()]
data = data[data.Content.notna()]
data['Content_punct'] = data['Content'].apply(lambda x:remove_punct(x.lower()))
data_final = SentimentAnalysisCountry(data, 'Content_punct', 0.05)
nltk_sentiment_df = get_value_counts(data_final, 'nltk_sentiment')
nltk_sentiment_df

,sentiment,counts,percentage
0,Negative,276,25.96
1,Neutral,6,0.56
2,Positive,781,73.47


In [41]:
#NAMIBIA
data = data_vac_NAM.copy()
data["Content"] = data[['Title', 'Storie']].apply(lambda x: ' '.join(x), axis=1)
data = data[data.Storie.notna()]
data = data[data.Content.notna()]
data['Content_punct'] = data['Content'].apply(lambda x:remove_punct(x.lower()))
data_final = SentimentAnalysisCountry(data, 'Content_punct', 0.05)
nltk_sentiment_df = get_value_counts(data_final, 'nltk_sentiment')
nltk_sentiment_df

,sentiment,counts,percentage
0,Negative,514,24.93
1,Neutral,5,0.24
2,Positive,1543,74.83


In [42]:
#NIGER
data = data_vac_NER.copy()
data["Content"] = data[['Title', 'Storie']].apply(lambda x: ' '.join(x), axis=1)
data = data[data.Storie.notna()]
data = data[data.Content.notna()]
data['Content_punct'] = data['Content'].apply(lambda x:remove_punct(x.lower()))
data_final = SentimentAnalysisCountry(data, 'Content_punct', 0.05)
nltk_sentiment_df = get_value_counts(data_final, 'nltk_sentiment')
nltk_sentiment_df

,sentiment,counts,percentage
0,Negative,1679,27.47
1,Neutral,25,0.41
2,Positive,4408,72.12


In [43]:
#NIGERIA
data = data_vac_NGA.copy()
data["Content"] = data[['Title', 'Storie']].apply(lambda x: ' '.join(x), axis=1)
data = data[data.Storie.notna()]
data = data[data.Content.notna()]
data['Content_punct'] = data['Content'].apply(lambda x:remove_punct(x.lower()))
data_final = SentimentAnalysisCountry(data, 'Content_punct', 0.05)
nltk_sentiment_df = get_value_counts(data_final, 'nltk_sentiment')
nltk_sentiment_df

,sentiment,counts,percentage
0,Negative,1651,27.46
1,Neutral,25,0.42
2,Positive,4336,72.12


In [44]:
#RWANDA
data = data_vac_RWA.copy()
data["Content"] = data[['Title', 'Storie']].apply(lambda x: ' '.join(x), axis=1)
data = data[data.Storie.notna()]
data = data[data.Content.notna()]
data['Content_punct'] = data['Content'].apply(lambda x:remove_punct(x.lower()))
data_final = SentimentAnalysisCountry(data, 'Content_punct', 0.05)
nltk_sentiment_df = get_value_counts(data_final, 'nltk_sentiment')
nltk_sentiment_df

,sentiment,counts,percentage
0,Negative,341,18.98
1,Neutral,12,0.67
2,Positive,1444,80.36


In [45]:
#SÃO TOMÉ AND PRÍNCIPE
data = data_vac_STP.copy()
data["Content"] = data[['Title', 'Storie']].apply(lambda x: ' '.join(x), axis=1)
data = data[data.Storie.notna()]
data = data[data.Content.notna()]
data['Content_punct'] = data['Content'].apply(lambda x:remove_punct(x.lower()))
data_final = SentimentAnalysisCountry(data, 'Content_punct', 0.05)
nltk_sentiment_df = get_value_counts(data_final, 'nltk_sentiment')
nltk_sentiment_df

NameError: name 'data_vac_STP' is not defined

In [46]:
#SENEGAL
data = data_vac_SEN.copy()
data["Content"] = data[['Title', 'Storie']].apply(lambda x: ' '.join(x), axis=1)
data = data[data.Storie.notna()]
data = data[data.Content.notna()]
data['Content_punct'] = data['Content'].apply(lambda x:remove_punct(x.lower()))
data_final = SentimentAnalysisCountry(data, 'Content_punct', 0.05)
nltk_sentiment_df = get_value_counts(data_final, 'nltk_sentiment')
nltk_sentiment_df

,sentiment,counts,percentage
0,Negative,114,15.30
1,Neutral,2,0.27
2,Positive,629,84.43


In [47]:
#SEYCHELLES
data = data_vac_SYC.copy()
data["Content"] = data[['Title', 'Storie']].apply(lambda x: ' '.join(x), axis=1)
data = data[data.Storie.notna()]
data = data[data.Content.notna()]
data['Content_punct'] = data['Content'].apply(lambda x:remove_punct(x.lower()))
data_final = SentimentAnalysisCountry(data, 'Content_punct', 0.05)
nltk_sentiment_df = get_value_counts(data_final, 'nltk_sentiment')
nltk_sentiment_df

,sentiment,counts,percentage
0,Negative,40,11.11
1,Neutral,2,0.56
2,Positive,318,88.33


In [48]:
#SIERRA LEONE
data = data_vac_SLE.copy()
data["Content"] = data[['Title', 'Storie']].apply(lambda x: ' '.join(x), axis=1)
data = data[data.Storie.notna()]
data = data[data.Content.notna()]
data['Content_punct'] = data['Content'].apply(lambda x:remove_punct(x.lower()))
data_final = SentimentAnalysisCountry(data, 'Content_punct', 0.05)
nltk_sentiment_df = get_value_counts(data_final, 'nltk_sentiment')
nltk_sentiment_df

,sentiment,counts,percentage
0,Negative,76,22.29
1,Neutral,2,0.59
2,Positive,263,77.13


In [49]:
#SOUTH AFRICA
data = data_vac_ZAF.copy()
data["Content"] = data[['Title', 'Storie']].apply(lambda x: ' '.join(x), axis=1)
data = data[data.Storie.notna()]
data = data[data.Content.notna()]
data['Content_punct'] = data['Content'].apply(lambda x:remove_punct(x.lower()))
data_final = SentimentAnalysisCountry(data, 'Content_punct', 0.05)
nltk_sentiment_df = get_value_counts(data_final, 'nltk_sentiment')
nltk_sentiment_df

,sentiment,counts,percentage
0,Negative,1834,26.87
1,Neutral,60,0.88
2,Positive,4932,72.25


In [50]:
#SOUTH SUDAN
data = data_vac_SSD.copy()
data["Content"] = data[['Title', 'Storie']].apply(lambda x: ' '.join(x), axis=1)
data = data[data.Storie.notna()]
data = data[data.Content.notna()]
data['Content_punct'] = data['Content'].apply(lambda x:remove_punct(x.lower()))
data_final = SentimentAnalysisCountry(data, 'Content_punct', 0.05)
nltk_sentiment_df = get_value_counts(data_final, 'nltk_sentiment')
nltk_sentiment_df

,sentiment,counts,percentage
0,Negative,178,33.46
1,Neutral,1,0.19
2,Positive,353,66.35


In [51]:
#ESWATINI
data = data_vac_SZ.copy()
data["Content"] = data[['Title', 'Storie']].apply(lambda x: ' '.join(x), axis=1)
data = data[data.Storie.notna()]
data = data[data.Content.notna()]
data['Content_punct'] = data['Content'].apply(lambda x:remove_punct(x.lower()))
data_final = SentimentAnalysisCountry(data, 'Content_punct', 0.05)
nltk_sentiment_df = get_value_counts(data_final, 'nltk_sentiment')
nltk_sentiment_df

,sentiment,counts,percentage
0,Negative,105,35.84
1,Neutral,3,1.02
2,Positive,185,63.14


In [52]:
#TOGO
data = data_vac_TGO.copy()
data["Content"] = data[['Title', 'Storie']].apply(lambda x: ' '.join(x), axis=1)
data = data[data.Storie.notna()]
data = data[data.Content.notna()]
data['Content_punct'] = data['Content'].apply(lambda x:remove_punct(x.lower()))
data_final = SentimentAnalysisCountry(data, 'Content_punct', 0.05)
nltk_sentiment_df = get_value_counts(data_final, 'nltk_sentiment')
nltk_sentiment_df

,sentiment,counts,percentage
0,Negative,50,20.0
1,Neutral,1,0.4
2,Positive,199,79.6


In [53]:
#UGANDA
data = data_vac_UGA.copy()
data["Content"] = data[['Title', 'Storie']].apply(lambda x: ' '.join(x), axis=1)
data = data[data.Storie.notna()]
data = data[data.Content.notna()]
data['Content_punct'] = data['Content'].apply(lambda x:remove_punct(x.lower()))
data_final = SentimentAnalysisCountry(data, 'Content_punct', 0.05)
nltk_sentiment_df = get_value_counts(data_final, 'nltk_sentiment')
nltk_sentiment_df

,sentiment,counts,percentage
0,Negative,835,33.12
1,Neutral,16,0.63
2,Positive,1670,66.24


In [54]:
#UNITED REPUBLIC OF TANZANIA
data = data_vac_TZA.copy()
data["Content"] = data[['Title', 'Storie']].apply(lambda x: ' '.join(x), axis=1)
data = data[data.Storie.notna()]
data = data[data.Content.notna()]
data['Content_punct'] = data['Content'].apply(lambda x:remove_punct(x.lower()))
data_final = SentimentAnalysisCountry(data, 'Content_punct', 0.05)
nltk_sentiment_df = get_value_counts(data_final, 'nltk_sentiment')
nltk_sentiment_df

,sentiment,counts,percentage
0,Negative,11,19.64
1,Positive,45,80.36


In [55]:
#ZAMBIA
data = data_vac_ZMB.copy()
data["Content"] = data[['Title', 'Storie']].apply(lambda x: ' '.join(x), axis=1)
data = data[data.Storie.notna()]
data = data[data.Content.notna()]
data['Content_punct'] = data['Content'].apply(lambda x:remove_punct(x.lower()))
data_final = SentimentAnalysisCountry(data, 'Content_punct', 0.05)
nltk_sentiment_df = get_value_counts(data_final, 'nltk_sentiment')
nltk_sentiment_df

,sentiment,counts,percentage
0,Negative,187,21.08
1,Neutral,3,0.34
2,Positive,697,78.58


In [56]:
#ZIMBABWE
data = data_vac_ZW.copy()
data["Content"] = data[['Title', 'Storie']].apply(lambda x: ' '.join(x), axis=1)
data = data[data.Storie.notna()]
data = data[data.Content.notna()]
data['Content_punct'] = data['Content'].apply(lambda x:remove_punct(x.lower()))
data_final = SentimentAnalysisCountry(data, 'Content_punct', 0.05)
nltk_sentiment_df = get_value_counts(data_final, 'nltk_sentiment')
nltk_sentiment_df

,sentiment,counts,percentage
0,Negative,978,22.97
1,Neutral,23,0.54
2,Positive,3256,76.49
